In [3]:


import pandas as pd
import replicaEVSE.sql_wrapper_functions as sql
import replicaEVSE.geography as geo
import os
import requests
import geopandas as gpd
%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

with open('../src/metadata/MySQLpwd.txt') as f:
    pw = f.readlines()
pw = pw[0]

In [4]:
# WA state
state = '53'
gdf = geo.get_bg_shapefiles('53')

In [5]:
len(gdf)
# WA state
state = '53'
gdf = geo.get_bg_shapefiles('53')
gdf.to_pickle('../../data/blockgroup_boundaries.pkl')

4783

In [6]:
gdf = pd.read_pickle('../../data/blockgroup_boundaries.pkl')

In [8]:
# Create list of blockgroup ids from which to pull data for 
# the simulation
bgrp_list = list(gdf.GEOID.values)

4783

In [12]:
2312.64/30


77.088